[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show1d/show1d_simple.ipynb)
# Show1D — Quick Demo
Interactive 1D viewer for spectra, line profiles, and time series.

In [1]:
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [2]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

env: ANYWIDGET_HMR=1


In [3]:
import numpy as np, torch
import quantem.widget
from quantem.widget import Show1D
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
def make_eels_spectrum(n=512, seed=0):
    torch.manual_seed(seed)
    e = torch.linspace(-20, 800, n, device=device)
    zlp = 1000 * torch.exp(-0.5 * (e / 3) ** 2)
    plasmon = 200 * torch.exp(-0.5 * ((e - 15) / 5) ** 2) + 80 * torch.exp(-0.5 * ((e - 30) / 8) ** 2)
    bg = torch.where(e > 5, 5000 * (e.clamp(min=5) / 5) ** -2.5, torch.tensor(0.0, device=device))
    edge = torch.where(e > 532, 15 * ((e - 532).clamp(min=0) / 50) ** 0.4 * torch.exp(-(e - 532) / 200), torch.tensor(0.0, device=device))
    spec = (zlp + plasmon + bg + edge).cpu().numpy()
    return e.cpu().numpy().astype(np.float32), (spec + np.random.default_rng(seed).poisson(2, n)).astype(np.float32)
def make_convergence_curve(n=200, seed=0):
    torch.manual_seed(seed)
    epochs = torch.arange(n, dtype=torch.float32, device=device)
    loss = (10.0 * torch.exp(-0.02 * epochs) + 0.1).cpu().numpy()
    return epochs.cpu().numpy(), (loss + 0.05 * np.random.default_rng(seed).standard_normal(n)).astype(np.float32)
print(f"Generators ready (device={device})")
print(f"quantem.widget {quantem.widget.__version__}")

Generators ready (device=mps)
quantem.widget 0.4.0a3


## Single EELS Spectrum

In [4]:
energy, spec = make_eels_spectrum()
Show1D(spec, x=energy, title="EELS Spectrum", x_label="Energy Loss", x_unit="eV", y_label="Counts")

Show1D(512 points)

## Auto-Contrast — Reveal Core-Loss Edges
The zero-loss peak dominates the Y-axis, hiding weak core-loss features.
Enable `auto_contrast=True` with percentile clipping to zoom into the signal.

In [5]:
Show1D(
    spec,
    x=energy,
    title="EELS — Auto-Contrast",
    x_label="Energy Loss",
    x_unit="eV",
    y_label="Counts",
    auto_contrast=True,
    percentile_low=2.0,
    percentile_high=98.0,
)

Show1D(512 points)

## Multiple Spectra Overlay
Compare spectra from different sample regions.

In [6]:
energy, spec1 = make_eels_spectrum(seed=0)
_, spec2 = make_eels_spectrum(seed=1)
_, spec3 = make_eels_spectrum(seed=2)
Show1D(
    [spec1, spec2, spec3],
    x=energy,
    labels=["Region A", "Region B", "Region C"],
    title="EELS — Multi-region Comparison",
    x_label="Energy Loss", x_unit="eV", y_label="Counts",
)

Show1D(3 traces x 512 points)

## Optimization Convergence
Track reconstruction error over iterations.

In [7]:
epochs, loss = make_convergence_curve()
Show1D(loss, x=epochs, title="Ptychography Convergence", x_label="Iteration", y_label="Error", log_scale=True)

Show1D(200 points)

## State Inspection

In [8]:
w = Show1D(spec, x=energy, title="EELS Spectrum", x_label="Energy Loss", x_unit="eV")
w.summary()

EELS Spectrum
Series:   1 x 512 points
Labels:   Data
X range:  -20 - 800
X axis:   Energy Loss (eV)
  [0] Data: mean=40.28  min=7.794e-06  max=3850  std=225.8
Display:  linear | grid
